In [1]:
class Foo:
    pass


x = Foo()

print(f"type(x): {type(x)}")
print(f"type(Foo): {type(Foo)}")

type(x): <class '__main__.Foo'>
type(Foo): <class 'type'>


In [2]:
for t in (int, float, dict, list, tuple):
    print(f"type({t}): {type(t)}")

type(<class 'int'>): <class 'type'>
type(<class 'float'>): <class 'type'>
type(<class 'dict'>): <class 'type'>
type(<class 'list'>): <class 'type'>
type(<class 'tuple'>): <class 'type'>


# Example 1

Creating a class 'Foo' via metaclass 'type'

In [3]:
Foo = type('Foo', (), {})

x = Foo()
x

In [4]:
class Foo:
    pass

x = Foo()
x

# Example 2

Here, <bases> is a tuple with a single element Foo, specifying the parent class that Bar inherits from. An attribute, attr, is initially placed into the namespace dictionary:

In [5]:
Bar = type('Bar', (Foo,), dict(attr=100))
x = Bar()

print(x.attr)
print(x.__class__)
print(x.__class__.__bases__)

100
<class '__main__.Bar'>
(<class '__main__.Foo'>,)


In [6]:
class Bar(Foo):
    attr = 100


x = Bar()

print(x.attr)
print(x.__class__)
print(x.__class__.__bases__)

100
<class '__main__.Bar'>
(<class '__main__.Foo'>,)


# Example 3

This time, <bases> is again empty. Two objects are placed into the namespace dictionary via the <dct> argument. The first is an attribute named attr and the second a function named attr_val, which becomes a method of the defined class:

In [7]:
Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': lambda x : x.attr
    }
)

x = Foo()

print(x.attr)
print(x.attr_val())

100
100


In [8]:
class Foo:
    attr = 100
    def attr_val(self):
        return self.attr


x = Foo()

print(x.attr)
print(x.attr_val())

100
100


# Example 4

Only very simple functions can be defined with lambda in Python. In the following example, a slightly more complex function is defined externally then assigned to attr_val in the namespace dictionary via the name f:

In [13]:
def f(obj):
    print('attr =', obj.attr)


Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': f
    }
)

x = Foo()

print(x.attr)
x.attr_val()

100
attr = 100


In [14]:
def f(obj):
    print('attr =', obj.attr)


class Foo:
    attr = 100
    attr_val = f


x = Foo()

print(x.attr)
x.attr_val()

100
attr = 100


In the following, a custom method called new() is defined and assigned as the __new__() method for Foo:

In [18]:
def new(cls):
    x = object.__new__(cls)
    x.attr = 100
    return x

Foo.__new__ = new

f = Foo()
print(f.attr)

g = Foo()
print(g.attr)

100
100


# Defining own metaclass

In [19]:
# Spoiler alert:  This doesn't work!
def new(cls):
    x = type.__new__(cls)
    x.attr = 100
    return x

type.__new__ = new

TypeError: can't set attributes of built-in/extension type 'type'

In [1]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x

In [2]:
class Foo(metaclass=Meta):
    pass

Foo.attr

100

In [3]:
class Bar(metaclass=Meta):
    pass

class Qux(metaclass=Meta):
    pass

Bar.attr, Qux.attr


(100, 100)

# Object Factory

In [4]:
class Foo:
    def __init__(self):
        self.attr = 100


x = Foo()
print(x.attr)


y = Foo()
print(y.attr)


z = Foo()
print(z.attr)

100
100
100


# Class Factory

In [5]:
class Meta(type):
    def __init__(
        cls, name, bases, dct
    ):
        cls.attr = 100

class X(metaclass=Meta):
    pass

print(X.attr)


class Y(metaclass=Meta):
    pass

print(Y.attr)


class Z(metaclass=Meta):
    pass

print(Z.attr)

100
100
100


# Is This Really Necessary?

In Python, there are at least a couple other ways in which effectively the same thing can be accomplished:

### Simple Inheritance

In [6]:
class Base:
    attr = 100


class X(Base):
    pass


class Y(Base):
    pass


class Z(Base):
    pass


print(X.attr)
print(Y.attr)
print(Z.attr)

100
100
100


### Class Decorators

In [7]:
def decorator(cls):
    class NewClass(cls):
        attr = 100
    return NewClass

@decorator
class X:
    pass

@decorator
class Y:
    pass

@decorator
class Z:
    pass


print(X.attr)
print(Y.attr)
print(Z.attr)

100
100
100
